In [1]:
from algorithms.activations import Linear, ReLU
from algorithms.model_selection import DEFAULT_METRICS, KFold
from algorithms.nn import NeuralNetwork
from algorithms.preprocessing import load_and_preprocess_dataset

In [2]:
X_train, y_train, X_test, y_test = load_and_preprocess_dataset()

In [3]:
k = 5
model = NeuralNetwork([(23, 16, ReLU), (16, 1, Linear)])

In [4]:
def lr_schedule(epoch):
    if epoch < 100:
        return 5e-5
    elif epoch < 200:
        return 1e-5
    elif epoch < 400:
        return 5e-6
    elif epoch < 800:
        return 1e-6
    else:
        return 5e-7

In [5]:
kfold = KFold(k=k, model=model).cv(X_train, y_train, batch_size=32, epochs=1000, alpha=1e-5)

In [6]:
train_scores, valid_scores = kfold.get_scores(as_dataframe=True)

In [7]:
print(f'{k}-fold cross validation scores on train dataset:')
train_scores

5-fold cross validation scores on train dataset:


,MSE,MAE,MAPE,R2
mean,0.221477,0.292905,2.143256,0.778326
std,0.032161,0.024305,0.613995,0.031779


In [8]:
print(f'{k}-fold cross validation scores on validation dataset:')
valid_scores

5-fold cross validation scores on validation dataset:


,MSE,MAE,MAPE,R2
mean,0.223511,0.291779,1.712177,0.781909
std,0.062496,0.031417,0.939085,0.045566


In [9]:
model = kfold.get_best_models()[0]

In [10]:
y_test_pred = model.predict(X_test)

In [11]:
for metric, metric_fn in DEFAULT_METRICS.items():
    print(f'{metric}: {metric_fn(y_test, y_test_pred)}')

MSE: 0.8011842676669235
MAE: 0.6614837121566636
MAPE: 1.7390162335511563
R2: 0.4430353568424834
